In [41]:
 # Dependencies
import requests
import pandas as pd
import json
from pprint import pprint
from pandas.io.json import json_normalize

In [42]:
# Create dataframe of JSON data
population_DF = pd.read_csv('census_data.csv',encoding='Latin-1')
population_DF.head()

,Id,Id2,Geography,Population Estimate (as of July 1) - 2017
0,0500000US01001,1001,"Autauga County, Alabama",55504
1,0500000US01003,1003,"Baldwin County, Alabama",212628
2,0500000US01005,1005,"Barbour County, Alabama",25270
3,0500000US01007,1007,"Bibb County, Alabama",22668
4,0500000US01009,1009,"Blount County, Alabama",58013


In [44]:
# Split NAME variable into two columns
counties = population_DF['Geography'].str.split(',', expand = True)

# Create columns for these split values of County and State
population_DF['County'] = counties[0]
population_DF['State'] = counties[1]

# Drop the full Id2 and Geography columns
population_DF = population_DF.drop(['Id2'], axis=1)
population_DF = population_DF.drop(['Geography'], axis=1)
population_DF.head()

,Id,Population Estimate (as of July 1) - 2017,County,State
0,0500000US01001,55504,Autauga County,Alabama
1,0500000US01003,212628,Baldwin County,Alabama
2,0500000US01005,25270,Barbour County,Alabama
3,0500000US01007,22668,Bibb County,Alabama
4,0500000US01009,58013,Blount County,Alabama


In [46]:
population_DF = population_DF.rename(columns={'Population Estimate (as of July 1) - 2017': 'Population'})
population_DF.head()

,Id,Population,County,State
0,0500000US01001,55504,Autauga County,Alabama
1,0500000US01003,212628,Baldwin County,Alabama
2,0500000US01005,25270,Barbour County,Alabama
3,0500000US01007,22668,Bibb County,Alabama
4,0500000US01009,58013,Blount County,Alabama


In [49]:
# Dropping Puerto Rico, Alaska, and Hawaii from dataframe because they will not be used
population_DF = population_DF[~population_DF['State'].str.contains('Puerto Rico')]
population_DF = population_DF[~population_DF['State'].str.contains('Hawaii')]
population_DF = population_DF[~population_DF['State'].str.contains('Alaska')]

In [50]:
# Open json file and save to variable geojson
with open('cb_2017_us_county_500k.json') as f:
    geojson = json.load(f)

In [51]:
# Utilize json_normalize to extract features from geoJSON
county_lines_DF = json_normalize(geojson["features"])
county_lines_DF.head()

,geometry.coordinates,geometry.type,properties.AFFGEOID,properties.ALAND,properties.AWATER,properties.COUNTYFP,properties.COUNTYNS,properties.GEOID,properties.LSAD,properties.NAME,properties.STATEFP,type
0,"[[[-85.657668, 31.880274999999997], [-85.65600...",Polygon,0500000US01005,2292144656,50538698,005,00161528,01005,06,Barbour,01,Feature
1,"[[[-88.473227, 31.893856], [-88.468879, 31.930...",Polygon,0500000US01023,2365869837,19144469,023,00161537,01023,06,Choctaw,01,Feature
2,"[[[-86.90589899999999, 31.753034999999997], [-...",Polygon,0500000US01035,2201948618,6643480,035,00161543,01035,06,Conecuh,01,Feature
3,"[[[-86.374974, 32.75358], [-86.33676799999999,...",Polygon,0500000US01051,1601762124,99965171,051,00161551,01051,06,Elmore,01,Feature
4,"[[[-87.71570899999999, 33.006824], [-87.641506...",Polygon,0500000US01065,1667907107,32423356,065,00161558,01065,06,Hale,01,Feature


In [59]:
mini_county_lines_DF = county_lines_DF[['properties.AFFGEOID',
                                        'geometry.type',
                                        'geometry.coordinates']]
mini_county_lines_DF.head()

,properties.AFFGEOID,geometry.type,geometry.coordinates
0,0500000US01005,Polygon,"[[[-85.657668, 31.880274999999997], [-85.65600..."
1,0500000US01023,Polygon,"[[[-88.473227, 31.893856], [-88.468879, 31.930..."
2,0500000US01035,Polygon,"[[[-86.90589899999999, 31.753034999999997], [-..."
3,0500000US01051,Polygon,"[[[-86.374974, 32.75358], [-86.33676799999999,..."
4,0500000US01065,Polygon,"[[[-87.71570899999999, 33.006824], [-87.641506..."


In [60]:
mini_county_lines_DF = mini_county_lines_DF.rename(columns={'properties.AFFGEOID': 'Id',
                                                            'geometry.type': 'Type',
                                                            'geometry.coordinates': 'Coordinates'})
mini_county_lines_DF.head()

,Id,Type,Coordinates
0,0500000US01005,Polygon,"[[[-85.657668, 31.880274999999997], [-85.65600..."
1,0500000US01023,Polygon,"[[[-88.473227, 31.893856], [-88.468879, 31.930..."
2,0500000US01035,Polygon,"[[[-86.90589899999999, 31.753034999999997], [-..."
3,0500000US01051,Polygon,"[[[-86.374974, 32.75358], [-86.33676799999999,..."
4,0500000US01065,Polygon,"[[[-87.71570899999999, 33.006824], [-87.641506..."


In [63]:
complete_DF = population_DF.merge(mini_county_lines_DF, on='Id')
complete_DF.head()

,Id,Population,County,State,Type,Coordinates
0,0500000US01001,55504,Autauga County,Alabama,Polygon,"[[[-86.91759499999999, 32.664169], [-86.91461,..."
1,0500000US01003,212628,Baldwin County,Alabama,Polygon,"[[[-88.0089856789154, 30.6834908939434], [-88...."
2,0500000US01005,25270,Barbour County,Alabama,Polygon,"[[[-85.657668, 31.880274999999997], [-85.65600..."
3,0500000US01007,22668,Bibb County,Alabama,Polygon,"[[[-87.421936, 33.003378999999995], [-87.31853..."
4,0500000US01009,58013,Blount County,Alabama,Polygon,"[[[-86.963358, 33.858221], [-86.959667, 33.857..."
